In [32]:
import json
import pennylane as qml
import pennylane.numpy as np
import scipy

In [33]:
def abs_dist(rho, sigma):
    """A function to compute the absolute value |rho - sigma|."""
    polar = scipy.linalg.polar(rho - sigma)
    return polar[1]

def word_dist(word):
    """A function which counts the non-identity operators in a Pauli word"""
    return sum(word[i] != "I" for i in range(len(word)))


# Produce the Pauli density for a given Pauli word and apply noise

def noisy_Pauli_density(word, lmbda):
    """
       A subcircuit which prepares a density matrix (I + P)/2**n for a given Pauli
       word P, and applies depolarizing noise to each qubit. Nothing is returned.

    Args:
            word (str): A Pauli word represented as a string with characters I,  X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.
    """
    
    i=0
    X=np.zeros((2,2),dtype=complex)
    X[0,1]=1
    X[1,0]=1
    Z=np.zeros((2,2),dtype=complex)
    Z[0,0]=1
    Z[1,1]=-1
    Y=np.zeros((2,2),dtype=complex)
    Y[0,1]=-1j
    Y[1,0]=1j
    
    if word[0]=='X':
        string_P=X
    elif word[0]=='Y':
        string_P=Y
    elif word[0]=='Z':
        string_P=Z
    else:
        string_P=np.eye(2)
    for  stri in word[1:]:
        if stri=='X':
            string_P=np.kron(string_P,X)
        elif stri=='Y':
            string_P=np.kron(string_P,Y)
        elif stri=='Z':
            string_P=np.kron(string_P,Z)
        else:
            string_P=np.kron(string_P,np.eye(2))
    Id=np.eye(2**len(word))
    Density_m=(Id+string_P)/(2**len(word))        
    qml.QubitDensityMatrix(Density_m,wires=list(range(len(word))))
    for i in range(len(word)):
        qml.DepolarizingChannel(lmbda,wires=i)
        
        

In [45]:
# Compute the trace distance from a noisy Pauli density to the maximally mixed density

def maxmix_trace_dist(word, lmbda):
    """
       A function compute the trace distance between a noisy density matrix, specified
       by a Pauli word, and the maximally mixed matrix.

    Args:
            word (str): A Pauli word represented as a string with characters I, X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.

    Returns:
            float: The trace distance between two matrices encoding Pauli words.
    """
    word_len=len(word)
    dev = qml.device( 'default.mixed',wires=word_len)
    
    @qml.qnode(dev)
    def circuit(word, lmbda):
        noisy_Pauli_density(word, lmbda)
        return qml.density_matrix(wires=list(range(word_len)))
    
    M=circuit(word, lmbda)-(np.eye(2**len(word))/(2**len(word)))  
    #np.trace(np.abs_dist(circuit(word, lmbda),circuit(word, 0)))
    A=scipy.linalg.sqrtm(np.dot(np.conjugate(np.transpose(M)),M))
    
    return 0.5*np.trace(A)#0.5*np.trace(A) 

In [50]:
def bound_verifier(word, lmbda):
    """
       A simple check function which verifies the trace distance from a noisy Pauli density
       to the maximally mixed matrix is bounded by (1 - lambda)^|P|.

    Args:
            word (str): A Pauli word represented as a string with characters I, X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.

    Returns:
            float: The difference between (1 - lambda)^|P| and T(rho_P(lambda), rho_0).
    """
    counts=0
    for i in word:
        if i=='I':
            continue
        counts=counts+1
    #print(maxmix_trace_dist(word, lmbda))
    #print((1 - lmbda)**(counts))
    return (1 - lmbda)**(counts)-maxmix_trace_dist(word, lmbda)

In [47]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:

    word, lmbda = json.loads(test_case_input)
    output = np.real(bound_verifier(word, lmbda))

    return str(output)


def check(solution_output: str, expected_output: str) -> None:

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your trace distance isn't quite right!"


In [48]:
test_cases = [['["XXI", 0.7]', '0.0877777777777777'], ['["XXIZ", 0.1]', '0.4035185185185055'], ['["YIZ", 0.3]', '0.30999999999999284'], ['["ZZZZZZZXXX", 0.1]', '0.22914458207245006']]


In [49]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '["XXI", 0.7]'...
(0.0022222222222223154617+0j)
0.09000000000000002
Correct!
Running test case 1 with input '["XXIZ", 0.1]'...
(0.32548148148149444459+0j)
0.7290000000000001
Correct!
Running test case 2 with input '["YIZ", 0.3]'...
(0.18000000000000709877+0j)
0.48999999999999994
Correct!
Running test case 3 with input '["ZZZZZZZXXX", 0.1]'...
(0.11953385802755002454+0j)
0.3486784401000001
Correct!
